In [1]:
import vrep # access all the VREPelements
import sys
import time
import signal
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
from math import sin, cos, atan2, sqrt

In [2]:
from robot import Robot
r = Robot()

Connected to remoteApi server.
 Pioneer_p3dx_ultrasonicSensor1 connected.
 Pioneer_p3dx_ultrasonicSensor2 connected.
 Pioneer_p3dx_ultrasonicSensor3 connected.
 Pioneer_p3dx_ultrasonicSensor4 connected.
 Pioneer_p3dx_ultrasonicSensor5 connected.
 Pioneer_p3dx_ultrasonicSensor6 connected.
 Pioneer_p3dx_ultrasonicSensor7 connected.
 Pioneer_p3dx_ultrasonicSensor8 connected.
 Pioneer_p3dx_ultrasonicSensor9 connected.
 Pioneer_p3dx_ultrasonicSensor10 connected.
 Pioneer_p3dx_ultrasonicSensor11 connected.
 Pioneer_p3dx_ultrasonicSensor12 connected.
 Pioneer_p3dx_ultrasonicSensor13 connected.
 Pioneer_p3dx_ultrasonicSensor14 connected.
 Pioneer_p3dx_ultrasonicSensor15 connected.
 Pioneer_p3dx_ultrasonicSensor16 connected.
 Vision sensor connected.
 Laser connected.
 Left motor connected.
 Right motor connected.
 Robot connected.


In [3]:
# integral part of PID
i_e = 0
# used in derivative part of PID
old_e = 0
def PID(k_p, k_i, k_d, e):
    # e = error
    global i_e, old_e
    i_e = i_e + e
    res = k_p*e + k_i*i_e + k_d*(e - old_e) 
    old_e = e
    return res

In [4]:
def foundBestDistance(best_distance, end_pos):
    xe, ye = end_pos
    z = r.get_current_position(r.robot_handle,-1)
    xn = z[0]
    yn = z[1]
    tmpd = sqrt((xe-xn)**2+(ye-yn)**2)
    if tmpd < best_distance:
        return tmpd, True
    return best_distance, False

# def isWalls():
#     distances = r.read_sonic_sensors()
#     if(distances[0] < 0.2 or distances[1] < 0.2 or distances[2] < 0.4 or distances[3] < 0.3 or distances[4] < 0.3 or distances[15]<0.2):
#         return True
#     return False
    
    
def stateMachine(end_pos):

    xi, yi, _ = r.collect_ground_truth()
    xe, ye = end_pos
    stop = False
    duration = 5
    best_distance = sqrt((xe-xi)**2+(ye-yi)**2)

    while(sqrt((xi-xe)**2 + (yi-ye)**2) > 0.1):
        
        z = r.get_current_position(r.robot_handle,-1)
        xi = z[0]
        yi = z[1]
        angle_old = r.get_current_orientation(r.robot_handle,-1)
        angle_d = atan2(ye - yi, xe - xi)
        angle = PID(0.75, 0, 0.1, angle_d - angle_old[2])
        r.set_velocity(0.3, angle)
        time.sleep(0.3)
        
        stop = False
        while(r.isWall() == True and stop == False):
            r.follow_wall_fuzzy(3)
            best_distance, stop = foundBestDistance(best_distance, end_pos)
            print(stop)

In [5]:
from collections import defaultdict 

# Class to represent a graph
class Graph: 
  
    def __init__(self, vertices): 
        self.V = vertices # No. of vertices 
        self.graph = [] # default dictionary to store graph
        self.pos   = []*vertices
        self.passed = [False]*self.V
       
    def addAllPos(self, pos):
        self.pos = pos
        
    # function to add an edge to graph 
    def addEdge(self, u, v):
        pos = self.pos
        w = sqrt((pos[u][0] - pos[v][0])**2 + (pos[u][1] - pos[v][1])**2)
        self.graph.append([u, v, w])
        self.graph.append([v, u, w])
      
    # The main function that finds shortest distances from src to 
    # all other vertices using Bellman-Ford algorithm.  The function 
    # also detects negative weight cycle 
    def BellmanFord(self, src): 
  
        # Step 1: Initialize distances from src to all other vertices 
        # as INFINITE 
        dist = [float("Inf")] * self.V 
        dist[src] = 0
        path = [0]*self.V
        # Step 2: Relax all edges |V| - 1 times. A simple shortest  
        # path from src to any other vertex can have at-most |V| - 1  
        # edges 
        for i in range(self.V - 1): 
            # Update dist value and parent index of the adjacent vertices of 
            # the picked vertex. Consider only those vertices which are still in 
            # queue 
            for u, v, w in self.graph: 
                if dist[u] != float("Inf") and dist[u] + w < dist[v]: 
                    dist[v] = dist[u] + w
                    path[v] = u
        return (path, dist)
    
    def verifyPointsPassed(self, path, dist, start):
        
        plan = []
        
        for i in range(0,self.V):
            if(self.passed[i] == True):
                dist[i] = -1
            
        max_x = np.argmax(dist)
        x = max_x
        plan.append(max_x)
        #print("Passed: "+str(x))
        while(path[x]!=start):
            self.passed[x] = True
            x = path[x]
            plan.append(x)
            #print("Passed: "+str(x))
        
        #print("Passed: "+str(path[x]))
        plan.append(start)
        self.passed[x] = True
        #print("---------")
        
        return (max_x, plan[::-1])
    
    def moveToVertices(self, plan):
        
        print(plan)
        for point in plan:
            print("Current position: ", str(r.collect_ground_truth()))
            print("Go to point: "+str(self.pos[point]))
            stateMachine(self.pos[point])
        print("------------")
    
    def pathPlanning(self):
            
        start  = 0
        self.passed[start] = True
        
        while((False in self.passed)):
            
            (path, dist) = self.BellmanFord(start)
            (max_x, plan) = self.verifyPointsPassed(path, dist, start)
            self.moveToVertices(plan)
            start = max_x
            
                

g = Graph(10)
g.addAllPos([[-2.0250,-0.8250], [3.8500,0.9000], [0.35000,-3.5000], [-3.05, -2.575], 
             [-5.075, -3.75], [1.3, 2.725], [4.05, 5.45], [-5.3, 0.65], [-5.775, 2.9], [-5.225, 5.55]])
g.addEdge(0, 1)
g.addEdge(0, 2)
g.addEdge(0, 5)
g.addEdge(0, 7)
g.addEdge(1, 2)
g.addEdge(1, 3)
g.addEdge(1, 5)
g.addEdge(2, 3)
g.addEdge(3, 4)
g.addEdge(5, 6)
g.addEdge(7, 8)
g.addEdge(8, 9)

# Print the solution 
g.pathPlanning();

[0, 2, 3, 4]
Current position:  (-2.0253775119781494, -0.8249915838241577, -5.129610872245394e-05)
Go to point: [-2.025, -0.825]
Current position:  (-2.0253775119781494, -0.8249915838241577, -5.129610872245394e-05)
Go to point: [0.35, -3.5]
Current position:  (0.349104106426239, -3.4987120628356934, -0.9443244338035583)
Go to point: [-3.05, -2.575]
Current position:  (-3.0279693603515625, -2.5778729915618896, 3.009296178817749)
Go to point: [-5.075, -3.75]
------------
[4, 3, 2, 0, 7, 8, 9]
Current position:  (-5.043054103851318, -3.741523265838623, -2.8193297386169434)
Go to point: [-5.075, -3.75]
Current position:  (-5.043054103851318, -3.741523265838623, -2.8193297386169434)
Go to point: [-3.05, -2.575]
Current position:  (-3.0505056381225586, -2.5773658752441406, 0.8114503622055054)
Go to point: [0.35, -3.5]
Current position:  (0.3528459966182709, -3.50114369392395, -0.41229140758514404)
Go to point: [-2.025, -0.825]
Current position:  (-2.0010764598846436, -0.8436527252197266, 2.4